In [154]:
import requests
import pandas as pd
import numpy as np

#maximum items to query at once is 1000
#implement a skip variable to loop over all transactions returning 1000 transactions for each iteration. -> While loop
def get_messarisubg_uniswap_withdraws(skip =0 ,limit=1000):

    url = "https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-ethereum"

    query = """{
  withdraws(skip: """ +str(skip)+""",first: """ +str(limit)+""", orderBy: timestamp, orderDirection: desc) {
    amountUSD
    blockNumber
    from
    hash
    to
    timestamp
    inputTokenAmounts
    inputTokens {
      decimals
      lastPriceUSD
      symbol
      name
    }
    pool {
      inputTokenBalances
      symbol
      totalValueLockedUSD
      fees {
        feePercentage
        feeType
      }
      name
  
    }
  }}"""
    headers = {"Content-Type": "application/json"} #define in the request https header the content type: we want to send a json query in the body (there are a total of 4 types of http header types)
    #HTTP POST is a request method of an Hypertext Transfer Protocol (HTTP) and is used to create or update a resource on the server. The HTTP POST method is used to submit web forms and files and images to the server.
    response = requests.post(url, headers=headers, json={"query": query}) #define in the http request body the json data we want to send to the server

    if response.status_code != 200:
        raise Exception("Failed to retrieve UniSwap exchanges: " + response.text)

    return response.json()

#get more than first 1000 rows
skip_counter = 0
list_of_df = []
while True:
    try: 

        result = get_messarisubg_uniswap_withdraws(skip = skip_counter)
        list_of_df.append(pd.json_normalize(result["data"]["withdraws"]))
        skip_counter += 1000
        
        
    except Exception: 
      break


df_withdraws = pd.concat(list_of_df).reset_index(drop = True)


In [155]:
#token 1 column flattening
inputTokens_df = pd.json_normalize(df_withdraws["inputTokens"])
inputToken1_df = pd.json_normalize(inputTokens_df[0])
inputToken2_df2 = pd.json_normalize(inputTokens_df[1])
#token 1 is leaving the pool
#token 2 is also leaving the pool -> we take away liquditiy!
token_info_df = pd.concat([inputToken1_df, inputToken2_df2], axis = 1)
token_info_df.columns = ["decimals_Token1", 'lastPriceUSD_Token1', 'symbol_Token1', 'name_Token1', 'decimals_Token2', 'lastPriceUSD_Token2', 'symbol_Token2', 'name_Token2']
token_info_df.reset_index (drop = True)
token_info_df





,decimals_Token1,lastPriceUSD_Token1,symbol_Token1,name_Token1,decimals_Token2,lastPriceUSD_Token2,symbol_Token2,name_Token2
0,18,1480.385046414694274099491143434891,WETH,Wrapped Ether,18,0,GRT,Graph Token
1,18,1480.385046414694274099491143434891,WETH,Wrapped Ether,18,0.2776882393505976926498828004995712,LOOKS,LooksRare Token
2,9,2.300540364957871621086783523775781,TONCOIN,Wrapped TON Coin,18,1480.385046414694274099491143434891,WETH,Wrapped Ether
3,18,1,DAI,Dai Stablecoin,18,0,T,Threshold Network Token
4,6,1,USDC,USD Coin,18,1480.385046414694274099491143434891,WETH,Wrapped Ether
...,...,...,...,...,...,...,...,...
5995,18,2.261039591369259810861147568628509,LDO,Lido DAO Token,18,1480.385046414694274099491143434891,WETH,Wrapped Ether
5996,6,1,USDC,USD Coin,18,1480.385046414694274099491143434891,WETH,Wrapped Ether
5997,6,1,USDC,USD Coin,18,1480.385046414694274099491143434891,WETH,Wrapped Ether
5998,18,1480.385046414694274099491143434891,WETH,Wrapped Ether,18,0,IMX,Immutable X


In [157]:
#Token Amounts being withdrawn
inputTokenAmounts_df = df_withdraws["inputTokenAmounts"]
inputTokenAmounts_df = inputTokenAmounts_df.apply(pd.Series) #basically the command pd.Series([[entry1,entry1], [entry2, entry2]]) -> the column is basically a list of lists and each list of the big list is a new column
inputTokenAmounts_df.columns = ["Amount_Token1", "Amount_Token2"]
inputTokenAmounts_df.reset_index(drop=True)

#Token Amounts in the Pool
inputTokenBalances_df = df_withdraws["pool.inputTokenBalances"]
inputTokenBalances_df = inputTokenBalances_df.apply(pd.Series)
inputTokenBalances_df.columns = ["pool.TokenBalance_Token1", "pool.TokenBalance_Token2"]
inputTokenBalances_df





,pool.TokenBalance_Token1,pool.TokenBalance_Token2
0,235995759586323275446,1221931076522739707802000
1,2109285269773080971755,11135512804337889061162784
2,684261913392861,450225573409362138154
3,1198925345638557275620,116313883394041335763182
4,92298954439184,66606856597369309391564
...,...,...
5995,900793985782820748184047,2139660648961890876768
5996,69637714460923,64327637532757366156980
5997,92298954439184,66606856597369309391564
5998,266928211144114758478,397665156462166962032584


In [158]:

fee_df = pd.json_normalize(df_withdraws["pool.fees"])
fe_0_df = pd.json_normalize(fee_df[0])
fe_1_df = pd.json_normalize(fee_df[1])
fe_2_df = pd.json_normalize(fee_df[2])
fee_df_new = pd.concat([fe_0_df,fe_1_df,fe_2_df], axis =1 )
fee_final_columns = [fee_df_new.iloc[0,1], fee_df_new.iloc[0,3], fee_df_new.iloc[0,5]]
fee_final_df = fee_df_new.drop("feeType", axis =1)
fee_final_df.columns = fee_final_columns
fee_final_df
print(fee_final_df)


,FIXED_LP_FEE,FIXED_PROTOCOL_FEE,FIXED_TRADING_FEE
0,0.3,0,0.3
1,0.3,0,0.3
2,1,0,1
3,0.3,0,0.3
4,0.05,0,0.05
...,...,...,...
5995,1,0,1
5996,0.3,0,0.3
5997,0.05,0,0.05
5998,1,0,1


In [159]:
#glue the dataframes together
df_withdraws_formatted = pd.concat([df_withdraws,inputTokenAmounts_df,token_info_df,inputTokenBalances_df,fee_final_df], axis =1).drop(["inputTokenAmounts", "inputTokens", "pool.inputTokenBalances", "pool.fees"], axis = 1)
df_withdraws_formatted.head()

,amountUSD,blockNumber,from,hash,to,timestamp,pool.symbol,pool.totalValueLockedUSD,pool.name,Amount_Token1,...,name_Token1,decimals_Token2,lastPriceUSD_Token2,symbol_Token2,name_Token2,pool.TokenBalance_Token1,pool.TokenBalance_Token2,FIXED_LP_FEE,FIXED_PROTOCOL_FEE,FIXED_TRADING_FEE
0,323.9945071672818246591423841970944,16619315,0x0e2c4be9f3408e5b1ff631576d946eb8c224b5ed,0x5ea30aca998bbc9a9f64e2e8fe091c18d664e6fc26af...,0xc36442b4a4522e871399cd717abdd847ab11fe88,1676285003,Wrapped Ether/Graph Token,349364.5935088702133042057395234099,Uniswap V3 Wrapped Ether/Graph Token 0.3%,218858267956674938,...,Wrapped Ether,18,0,GRT,Graph Token,235995759586323275446,1221931076522739707802000,0.3,0,0.3
1,159169.2485575865292431259382384477,16619307,0x4b5ab61593a2401b1075b90c04cbcdd3f87ce011,0x14f1376bffaad47413985f5084b8feee0e7d35d522b6...,0xc36442b4a4522e871399cd717abdd847ab11fe88,1676284907,Wrapped Ether/LooksRare Token,6214755.316897478477796294057950287,Uniswap V3 Wrapped Ether/LooksRare Token 0.3%,15236847904857642863,...,Wrapped Ether,18,0.2776882393505976926498828004995712,LOOKS,LooksRare Token,2109285269773080971755,11135512804337889061162784,0.3,0,0.3
2,43227.2433553090727563328199030286,16619306,0x4b62fa30fea125e43780dc425c2be5acb4ba743b,0xefcf5bcbbded04dcb0d962514bddd0b3e5aa182d4e5d...,0xc36442b4a4522e871399cd717abdd847ab11fe88,1676284895,Wrapped TON Coin/Wrapped Ether,2240679.35835228490123875827479846,Uniswap V3 Wrapped TON Coin/Wrapped Ether 1%,0,...,Wrapped TON Coin,18,1480.385046414694274099491143434891,WETH,Wrapped Ether,684261913392861,450225573409362138154,1,0,1
3,0,16619302,0xa9c154ee5411edf3ab09024a4a6136df70e9ef44,0x66dbbbbf7e956e626171a35208f05e368fed846999cf...,0x2d4cd5f988e962127e99d953f281bf15eca02c5a,1676284847,Dai Stablecoin/Threshold Network Token,1198.92534563855727562,Uniswap V3 Dai Stablecoin/Threshold Network To...,0,...,Dai Stablecoin,18,0,T,Threshold Network Token,1198925345638557275620,116313883394041335763182,0.3,0,0.3
4,205697.241974423877272051700261863,16619302,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,0x2ff4b2376a3e1fbabf5197a3409005e52a295891a1fe...,0xc36442b4a4522e871399cd717abdd847ab11fe88,1676284847,USD Coin/Wrapped Ether,190902748.9346174506093136725434361,Uniswap V3 USD Coin/Wrapped Ether 0.05%,202581804594,...,USD Coin,18,1480.385046414694274099491143434891,WETH,Wrapped Ether,92298954439184,66606856597369309391564,0.05,0,0.05


In [147]:
int(df_withdraws_formatted["Amount_Token1"][0])/(10**df_withdraws_formatted["decimals_Token1"][0])


5472.398987800907

In [152]:
float(df_withdraws_formatted["lastPriceUSD_Token1"][0])

1.2899771448070623

In [153]:
df_withdraws_formatted["hash"][0]

'0xca0405a5d3a977e7e017fc80a51e97a004a5a6663c1c41489728713d05712537'